# TRATAMENTO DOS DADOS DO CNES, SIGTAP E TETO

O tratamento 

In [ ]:
#Pandas: Possui inúmeras funções e comandos para importar arquivos, analisar dados, tratar dados, etc.
import pandas as pd # importando a biblioteca pandas

#Numpy: Possui uma série de funções e comandos para trabalharmos com números de forma em geral(formatação, calculos, etc)
import numpy as np # importando a biblioteca numpy

#Time: Possui uma série de funções e comandos para trabalharmos com tempo
import time # importando a biblioteca time

#Zipfile: Possui uma série de funções e comandos para trabalharmos com arquivos zip
import zipfile # importando a biblioteca zipfile

#shutil: Possui uma série de funções e comandos para trabalharmos com arquivos
import shutil # Biblioteca para manipular arquivos e pastas

# urllib.request: Possui uma série de funções e comandos para trabalharmos com arquivos
import urllib.request # Biblioteca para fazer download de arquivos

#openpyxl: Possui uma série de funções e comandos para trabalharmos com arquivos excel
import openpyxl # Biblioteca para manipular arquivos excel

#re: Possui uma série de funções e comandos para trabalharmos com expressões regulares
import re # importando a biblioteca re

#os: Possui uma série de funções e comandos para trabalharmos com arquivos e diretórios
import os # importando a biblioteca os

#ftplib: Possui uma série de funções e comandos para trabalharmos com arquivos
import ftplib # Biblioteca para fazer download de arquivos

#Warnings: Possui detalhes sobre os avisos e alertas que aparecem, porém podemos utiliza-lo também para que os alertas de
#futuras atualizações e metodos depreciados não sejam exibidos
import warnings
warnings.filterwarnings("ignore") 
tempo_inicial = time.time() # tempo inicial para calcular o tempo de execução do código

from datetime import date,  timedelta # Biblioteca para manipular datas
from glob import glob # Utilizado para listar arquivos de um diretório
from selenium import webdriver # Biblioteca para automatizar o navegador
from selenium.webdriver.common.by import By
from datetime import date
from dateutil.relativedelta import relativedelta

#Comando para exibir todas colunas do arquivo
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# tempo de execução
start_time = time.time()

In [ ]:
# formatação de datas para o nome dos arquivos
data_atual = date.today() # Data atual
data_corrente = date.today().strftime('%Y%m') # Data atual no formato YYYYMM
data_passada = (date.today() - relativedelta(months=1)).strftime('%Y%m') # Data atual menos 1 mês no formato YYYYMM
data_passada_a = (date.today() - relativedelta(months=2)).strftime('%Y%m') # Data atual menos 2 mês no formato YYYYMM

In [ ]:
# Definição de pastas
pasta_entrada = 'BASE/' # Pasta onde estão os arquivos de entrada
pasta_saida = 'RESULTADO/' # Pasta onde serão gravados os arquivos de saída
pasta_planilha = 'PLANILHA/' # Pasta onde serão gravados os arquivos para ser analisados (planilha)

for pasta in [pasta_entrada, pasta_saida, pasta_planilha]:
    try:
        os.mkdir(pasta)
    except OSError:
        pass

    for arquivo in os.listdir(pasta):
        arquivo_pasta = os.path.join(pasta, arquivo)
        try:
            if os.path.isfile(arquivo_pasta) or os.path.islink(arquivo_pasta):
                os.unlink(arquivo_pasta) # deleta o arquivo
            elif os.path.isdir(arquivo_pasta):
                shutil.rmtree(arquivo_pasta) # deleta o diretório
        except Exception as e:
            print(f'Falha ao deletar {arquivo_pasta}. Motivo: {e}')
print(f"Criando as pastas =======================================>: {time.strftime('%H:%M:%S')}")

In [ ]:
# Download dos arquivos
url_c = f'ftp://ftp.datasus.gov.br/cnes/BASE_DE_DADOS_CNES_{data_passada}.ZIP'
filename_c = pasta_entrada + f'BASE_DE_DADOS_CNES_{data_passada}.ZIP'

try:
    with urllib.request.urlopen(url_c) as response, open(filename_c, 'wb') as out_file:
        data = response.read()  # Lê os dados do arquivo
        out_file.write(data)  # Escreve os dados em um arquivo local
except urllib.error.URLError:
    url_c = f'ftp://ftp.datasus.gov.br/cnes/BASE_DE_DADOS_CNES_{data_passada_a}.ZIP'
    filename_c = pasta_entrada + f'BASE_DE_DADOS_CNES_{data_passada_a}.ZIP'
    with urllib.request.urlopen(url_c) as response, open(filename_c, 'wb') as out_file:
        data = response.read()  # Lê os dados do arquivo
        out_file.write(data)  # Escreve os dados em um arquivo local

print(f"Download dos arquivos CNES ==============================>: {time.strftime('%H:%M:%S')}")

In [ ]:
ftp = ftplib.FTP('ftp2.datasus.gov.br') # Conecta ao servidor FTP
ftp.login() # Faz login no servidor FTP
ftp.cwd('/pub/sistemas/tup/downloads/') # Navega até o diretório desejado

arquivos = ftp.nlst() # Lista os arquivos disponíveis
arquivo = [a for a in arquivos if a.startswith(f'TabelaUnificada_{data_corrente}') and a.endswith('.zip')]

if arquivo:
    url_s = f'ftp://ftp2.datasus.gov.br/pub/sistemas/tup/downloads/{arquivo[0]}'
    filename_s = pasta_entrada + f'BASE_DE_DADOS_SIGTAP_{data_corrente}.ZIP'

    with urllib.request.urlopen(url_s) as response, open(filename_s, 'wb') as out_file:
        data = response.read()  # Lê os dados do arquivo
        out_file.write(data)  # Escreve os dados em um arquivo local

    print(f"Download dos arquivos SIGTAP ===========================>: {time.strftime('%H:%M:%S')}")
else:
    print(f"Nenhum arquivo SIGTAP encontrado! ======================>: {time.strftime('%H:%M:%S')}")


In [ ]:
url = 'https://www.gov.br/saude/pt-br/composicao/saes/saips/plano-atendimento-perf-cir-eletiva-vrs-4a.xlsx'
filename = pasta_entrada + f'BASE_DE_DADOS_SAIPS_V4.xlsx'

with urllib.request.urlopen(url) as response, open(filename, 'wb') as out_file:
    data = response.read()  # Lê os dados do arquivo
    out_file.write(data)  # Escreve os dados em um arquivo local

print(f"Download do arquivo BASE_DE_DADOS_SAIPS.xlsx ==============>: {time.strftime('%H:%M:%S')}")

In [ ]:
url_t = 'https://sismac.saude.gov.br/teto_financeiro_brasil#'
filename = pasta_entrada + f'BASE_DE_DADOS_TETO_FINANCEIRO_BRASIL.xlsx'


options = webdriver.EdgeOptions()
options.use_chromium = True
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Edge(executable_path=r'NAVEGADOR\msedgedriver.exe', options=options)
driver.get(url_t)

element = driver.find_element(By.XPATH, '//*[@id="conteudoPanel"]/div[3]/div/div/div[2]/p/a')
element.click()

driver.quit()

print(f"Download do arquivo BASE_DE_DADOS_TETO_MAC.xlsx ========>: {time.strftime('%H:%M:%S')}")

# Importação dos dados CNES, SIGTAP e TETO MAC: Analise e Tratamento de Dados

## BASE CNES

In [ ]:
dado_cnes = glob('BASE/BASE_DE_DADOS_CNES*.ZIP')[0] # Listando os arquivos do diretório

In [ ]:
with zipfile.ZipFile(dado_cnes, 'r') as myzip: # Abrindo o arquivo zip
   print(myzip.namelist(), sep=',') # Listando os arquivos do zip

In [ ]:
with zipfile.ZipFile(dado_cnes) as myzip: # Abrindo o arquivo zip
   with myzip.open('tbEstabelecimento202302.csv') as myfile:  
      df_cnes_servicos = pd.read_csv(myfile, sep=';', encoding='latin-1', low_memory=False , nrows=0) # Lendo o arquivo csv
df_cnes_servicos.columns.values # Exibindo os valores do arquivo

In [ ]:
colunas_c = ['CO_UNIDADE','CO_CNES','NO_RAZAO_SOCIAL','NO_FANTASIA','TP_UNIDADE','TP_GESTAO','CO_ESTADO_GESTOR','CO_MUNICIPIO_GESTOR','CO_MOTIVO_DESAB'] 
# Selecionando as colunas que serão utilizadas

In [ ]:
with zipfile.ZipFile(dado_cnes) as myzip: # Abrindo o arquivo zip
   with myzip.open('tbEstabelecimento202302.csv') as myfile:  
      df_cnes = pd.read_csv(myfile, sep=';', encoding='latin-1', low_memory=False , usecols=colunas_c) # Lendo o arquivo csv
df_cnes.head() # Exibindo as 5 primeiras linhas do arquivo

In [ ]:
df_cnes.info() # Exibindo informações do arquivo

### SERVIÇO X CLASS

In [ ]:
with zipfile.ZipFile(dado_cnes) as myzip: # Abrindo o arquivo zip
   with myzip.open('rlEstabServClass202302.csv') as myfile:  
      df_cnes_servicos = pd.read_csv(myfile, sep=';', encoding='latin-1', low_memory=False , nrows=0) # Lendo o arquivo csv
df_cnes_servicos.columns.values # Exibindo os valores do arquivo

In [ ]:
conlunas_s = ['CO_UNIDADE', 'CO_SERVICO', 'CO_CLASSIFICACAO'] # Selecionando as colunas que serão utilizadas

In [ ]:
with zipfile.ZipFile(dado_cnes) as myzip: # Abrindo o arquivo zip
   with myzip.open('rlEstabServClass202302.csv') as myfile:  
      df_cnes_servicos = pd.read_csv(myfile, sep=';', encoding='latin-1', low_memory=False , usecols=conlunas_s) # Lendo o arquivo csv
df_cnes_servicos.head() # Exibindo as 5 primeiras linhas do arquivo

In [ ]:
df_cnes_servicos.info() # Exibindo informações do arquivo

### HABILITAÇÃO  

In [ ]:
with zipfile.ZipFile(dado_cnes) as myzip: # Abrindo o arquivo zip
   with myzip.open('rlEstabSipac202302.csv') as myfile:  
      df_cnes_habilitacao = pd.read_csv(myfile, sep=';', encoding='latin-1', low_memory=False , nrows=0) # Lendo o arquivo csv
df_cnes_habilitacao.columns.values # Exibindo os valores do arquivo

In [ ]:
conlunas_h = ['CO_UNIDADE', 'COD_SUB_GRUPO_HABILITACAO'] # Selecionando as colunas que serão utilizadas

In [ ]:
with zipfile.ZipFile(dado_cnes) as myzip: # Abrindo o arquivo zip
   with myzip.open('rlEstabSipac202302.csv') as myfile:  
      df_cnes_habilitacao = pd.read_csv(myfile, sep=';', encoding='latin-1', low_memory=False , usecols=conlunas_h) # Lendo o arquivo csv
df_cnes_habilitacao.head() # Exibindo as 5 primeiras linhas do arquivo

In [ ]:
df_cnes_habilitacao.info()  # Exibindo informações do arquivo

### LEITOS E LEITOS DE UTI

In [ ]:
with zipfile.ZipFile(dado_cnes) as myzip: # Abrindo o arquivo zip
   with myzip.open('rlEstabComplementar202302.csv') as myfile:  
      df_cnes_leitos = pd.read_csv(myfile, sep=';', encoding='latin-1', low_memory=False , nrows=0) # Lendo o arquivo csv
df_cnes_leitos.columns.values # Exibindo os valores do arquivo

In [ ]:
colunas_l = ['CO_UNIDADE', 'CO_LEITO', 'CO_TIPO_LEITO','QT_EXIST','QT_SUS'] # Selecionando as colunas que serão utilizadas

In [ ]:
with zipfile.ZipFile(dado_cnes) as myzip: # Abrindo o arquivo zip
   with myzip.open('rlEstabComplementar202302.csv') as myfile:  
      df_cnes_leitos = pd.read_csv(myfile, sep=';', encoding='latin-1', low_memory=False , usecols=colunas_l) # Lendo o arquivo csv
df_cnes_leitos.head() # Exibindo as 5 primeiras linhas do arquivo

In [ ]:
df_cnes_leitos.info() # Exibindo informações do arquivo

#### NOME DAS HABILITAÇÕES E LEITOS

In [ ]:
with zipfile.ZipFile(dado_cnes) as myzip: # Abrindo o arquivo zip
   with myzip.open('tbSubGruposHabilitacao202302.csv') as myfile:  
      df_cnes_habilitacao_d = pd.read_csv(myfile, sep=';', encoding='latin-1', low_memory=False , nrows=0) # Lendo o arquivo csv
df_cnes_habilitacao_d.columns.values # Exibindo os valores do arquivo

In [ ]:
colunas_h_d = ['CO_CODIGO_GRUPO', 'NO_DESCRICAO_GRUPO'] # Selecionando as colunas que serão utilizadas

In [ ]:
with zipfile.ZipFile(dado_cnes) as myzip: # Abrindo o arquivo zip
   with myzip.open('tbSubGruposHabilitacao202302.csv') as myfile:  
      df_cnes_habilitacao_d = pd.read_csv(myfile, sep=';', encoding='latin-1', low_memory=False , usecols=colunas_h_d) # Lendo o arquivo csv
df_cnes_habilitacao_d.head() # Exibindo as 5 primeiras linhas do arquivo

In [ ]:
df_cnes_habilitacao = pd.merge(df_cnes_habilitacao, df_cnes_habilitacao_d, left_on='COD_SUB_GRUPO_HABILITACAO', right_on='CO_CODIGO_GRUPO', how='left')
df_cnes_habilitacao.drop(['COD_SUB_GRUPO_HABILITACAO'], axis=1, inplace=True)
df_cnes_habilitacao.head() # Exibindo as 5 primeiras linhas do arquivo

### Merge entre os arquivos CNES

### Merge entre os arquivos CNES e Serviços 

In [ ]:
df_base_cnes_s = pd.merge(df_cnes, df_cnes_servicos, on='CO_UNIDADE', how='left') # Juntando os arquivos
df_base_cnes_s.head() # Exibindo as 5 primeiras linhas do arquivo

In [ ]:
df_base_cnes_s.drop_duplicates() # Removendo duplicados
df_base_cnes_s.info() # Exibindo informações do arquivo

### Merge entre os arquivos CNES e Habilitação  

In [ ]:
df_base_cnes_h = pd.merge(df_cnes, df_cnes_habilitacao, on='CO_UNIDADE', how='left') # Juntando os arquivos
df_base_cnes_h.head() # Exibindo as 5 primeiras linhas do arquivo

In [ ]:
df_base_cnes_h.drop_duplicates() # Removendo duplicados
df_base_cnes_h.info() # Exibindo informações do arquivo

In [ ]:
df_base_cnes_h.head() # Exibindo as 5 primeiras linhas do arquivo

### Merge entre os arquivos CNES e Leitos

In [ ]:
df_base_cnes_l = pd.merge(df_cnes, df_cnes_leitos, on='CO_UNIDADE', how='left') # Juntando os arquivos
df_base_cnes_l.head() # Exibindo as 5 primeiras linhas do arquivo

In [ ]:
df_base_cnes_l.drop_duplicates() # Removendo duplicados
df_base_cnes_l.info() # Exibindo informações do arquivo

### Salvando o banco de dados CNES em csv  

In [ ]:
df_base_cnes_s.to_csv('BASE\.BASE_CNES_SERVICOS.csv', sep=';', encoding='latin-1', index=False) # Exportando o arquivo para csv
df_base_cnes_h.to_csv('BASE\.BASE_CNES_HABILITACAO.csv', sep=';', encoding='latin-1', index=False) # Exportando o arquivo para csv
df_base_cnes_l.to_csv('BASE\.BASE_CNES_LEITOS.csv', sep=';', encoding='latin-1', index=False) # Exportando o arquivo para csv

In [ ]:
tempo_analise = time.time()
tempo_total = (float(tempo_analise - tempo_inicial)/60)
print(f"Tempo total de parcial:: {tempo_total:,.1f} minutos" )

## BASE TETO MAC

In [ ]:
dado_teto = glob(f'BASE-T/*_relatorio*.xlsx')[0] # Listando os arquivos do diretório
df_teto_mac = pd.read_excel(dado_teto, sheet_name='relatorioTetoFinanceiroBrasilEx') # Lê o arquivo excel
df_teto_mac.drop(df_teto_mac.loc[df_teto_mac['Descrição Gestão']=='Total UF'].index, inplace=True) # Remove as linhas com o valor 'Total'
df_teto_mac.head() # Exibe as 5 primeiras linhas do arquivo

In [ ]:
df_teto_mac.info() # Exibe informações do arquivo

In [ ]:
df_teto_mac.to_csv('BASE\.BASE_TETO_MAC.csv', sep=';', encoding='latin-1', index=False) # Exporta o arquivo para csv

In [ ]:
tempo_processo = time.time()
tempo_total = (float(tempo_processo - tempo_inicial)/60)
tempo_parcial = (float(tempo_processo - tempo_analise)/60)
print(f"Tempo total de parcial: {tempo_parcial:,.1f} minutos" )
print(f"Tempo total de execução: {tempo_total:,.1f} minutos" )

## BASE SIGTAP

In [ ]:
dado_sigtap = glob('BASE/BASE_DE_DADOS_SIGTAP*.zip')[0] # Listando os arquivos do diretório

In [ ]:
with zipfile.ZipFile(dado_sigtap) as myzip: # Abrindo o arquivo zip
   print(myzip.namelist(),sep=",") # Listando os arquivos do zip

## Importando tabela de procedimentos

In [ ]:
with zipfile.ZipFile(dado_sigtap) as myzip: # Abrindo o arquivo zip
   with myzip.open('tb_procedimento.txt') as myfile:  
      df_sigtap = pd.read_fwf(myfile, colspecs=[(0,10), (10,260), (260,261), (261,262), (262,266), (266,270), (270,274), (274,278), (278,282), (282,292), (292,302), (302,312), (312,314), (314,320), (320,324), (324,330)], 
                              names=["CO_PROCEDIMENTO", "NO_PROCEDIMENTO", "TP_COMPLEXIDADE","TP_SEXO","QT_MAXIMA_EXECUCAO","QT_DIAS_PERMANENCIA","QT_PONTOS","VL_IDADE_MINIMA","VL_IDADE_MAXIMA",
                                     "VL_SH","VL_SA","VL_SP","CO_FINANCIAMENTO","CO_RUBRICA","QT_TEMPO_PERMANENCIA","DT_COMPETENCIA"], encoding='latin') # Lendo o arquivo txt
df_sigtap.drop(['TP_COMPLEXIDADE','TP_SEXO','QT_MAXIMA_EXECUCAO','QT_DIAS_PERMANENCIA','QT_PONTOS','VL_IDADE_MINIMA','VL_IDADE_MAXIMA','VL_SH','VL_SA','VL_SP',
                'CO_FINANCIAMENTO','CO_RUBRICA','QT_TEMPO_PERMANENCIA','DT_COMPETENCIA'], axis=1, inplace=True) # Removendo colunas desnecessárias
df_sigtap.head() # Exibindo as 5 primeiras linhas do arquivo

### Importar serviço/classificação

In [ ]:
with zipfile.ZipFile(dado_sigtap) as myzip: # Abrindo o arquivo zip
   with myzip.open('rl_procedimento_servico.txt') as myfile:  
      df_sigtap_servico = pd.read_fwf(myfile, colspecs=[(0,10), (10,13), (13,16), (16,22)], 
                              names=["CO_PROCEDIMENTO","CO_SERVICO","CO_CLASSIFICACAO","DT_COMPETENCIA"], encoding='latin') # Lendo o arquivo txt
df_sigtap_servico.drop(['DT_COMPETENCIA'], axis=1, inplace=True) # Removendo colunas desnecessárias
df_sigtap_servico.head() # Exibindo as 5 primeiras linhas do arquivo

### Importar Habilitação

In [ ]:
with zipfile.ZipFile(dado_sigtap) as myzip: # Abrindo o arquivo zip
   with myzip.open('rl_procedimento_habilitacao.txt') as myfile:  
      df_sigtap_habilitacao = pd.read_fwf(myfile, colspecs=[(0,10), (10,14), (14,18), (18,24)], names=["CO_PROCEDIMENTO", "CO_HABILITACAO", "NU_GRUPO_HABILITACAO", "DT_COMPETENCIA"]) # Lendo o arquivo txt
df_sigtap_habilitacao.drop(['NU_GRUPO_HABILITACAO','DT_COMPETENCIA'], axis=1, inplace=True) # Removendo colunas desnecessárias
df_sigtap_habilitacao.head() # Exibindo as 5 primeiras linhas do arquivo

### Importar Registro

In [ ]:
with zipfile.ZipFile(dado_sigtap) as myzip: # Abrindo o arquivo zip
   with myzip.open('rl_procedimento_registro.txt') as myfile:  
      df_sigtap_modalidade = pd.read_fwf(myfile, colspecs=[(0,10), (10,12), (12,18)], names=["CO_PROCEDIMENTO", "CO_REGISTRO", "DT_COMPETENCIA"]) # Lendo o arquivo txt
df_sigtap_modalidade.drop(['DT_COMPETENCIA'], axis=1, inplace=True) # Removendo colunas desnecessárias
df_sigtap_modalidade.head() # Exibindo as 5 primeiras linhas do arquivo


In [ ]:
with zipfile.ZipFile(dado_sigtap) as myzip: # Abrindo o arquivo zip
   with myzip.open('tb_registro.txt') as myfile:  
      df_sigtap_registro = pd.read_fwf(myfile, colspecs=[(0,2), (2,52), (52,58)], names=["CO_REGISTRO", "NO_REGISTRO", "DT_COMPETENCIA"], encoding='latin') # Lendo o arquivo txt
df_sigtap_registro.drop(['DT_COMPETENCIA'], axis=1, inplace=True) # Removendo colunas desnecessárias
df_sigtap_registro.head() # Exibindo as 5 primeiras linhas do arquivo

df_sigtap_modalidade = pd.merge(df_sigtap_modalidade, df_sigtap_registro, on='CO_REGISTRO', how='left') # Juntando os arquivos
df_sigtap_modalidade.head() # Exibindo as 5 primeiras linhas do arquivo

### Definido Grupo, Sub-grupo e forma de organização

In [ ]:
df_planilha_proc = glob('BASE\BASE_DE_DADOS_SAIPS*.xlsx')[0] # Listando os arquivos do diretório

In [ ]:
df_planilha_proc = pd.read_excel(df_planilha_proc, sheet_name='PROCEDIMENTOS') # Lendo o arquivo excel
df_planilha_proc.head() # Exibindo as 5 primeiras linhas do arquivo

In [ ]:
df_planilha_proc.rename(columns={'Código do Procedimento':'CO_PROCEDIMENTO','Unnamed: 1':'SUBGRUPO','Unnamed: 2':'COD_PROCEDIMENTO', 'Unnamed: 3':'DEC_PROCEDIMENTO', 'Unnamed: 4':'codigo', 
                                 'Dados de Produção Brasil (2018 a 2022)':'QT_PROD_SIASUS', 'Unnamed: 6':'% ELET SIASUS', 'Unnamed: 7':'QT_PROD AIH 2018 a 2022',
                                 'Unnamed: 8':'% ELET AIH', 'Exigência SIGTAP':'EXIGE SERVIÇO', 'Unnamed: 10':'EXIGE HABILITACAO', 'Instrumento de REGISTRO':'BPA_I',
                                 'Unnamed: 12':'AIH', 'Unnamed: 13':'APAC'}, inplace=True) # Renomeando colunas
df_planilha_proc.drop(0, inplace=True) # Remove a primeira linha do arquivo
df_planilha_proc.drop(['codigo'], axis=1, inplace=True) # Remove a coluna codigo
df_planilha_proc.head() # Exibindo as 5 primeiras linhas do arquivo

### Merge entre os arquivos SIGTAP

In [ ]:
df_sigtap_f = pd.merge(df_planilha_proc, df_sigtap_modalidade, left_on='COD_PROCEDIMENTO', right_on='CO_PROCEDIMENTO', how='left') # Juntando os arquivos 3
df_sigtap_f.head() # Exibindo as 5 primeiras linhas do arquivo

In [ ]:
df_sigtap_f = pd.merge(df_sigtap_f, df_sigtap_servico, on='CO_PROCEDIMENTO', how='left') # Juntando os arquivos
df_sigtap_f.head() # Exibindo as 5 primeiras linhas do arquivo

In [ ]:
df_sigtap_f = pd.merge(df_sigtap_f, df_sigtap_habilitacao, on='CO_PROCEDIMENTO', how='left') # Juntando os arquivos
df_sigtap_f.head() # Exibindo as 5 primeiras linhas do arquivo

In [ ]:
df_sigtap_f.drop(['CO_PROCEDIMENTO'], axis=1, inplace=True) # Removendo colunas desnecessárias
df_sigtap_f.info() # Exibindo informações do arquivo

## Salvar Banco de dados SIGTAP 

In [ ]:
df_sigtap_f.to_csv('BASE\.BASE_SIGTAP_GERAL.csv', sep=';', encoding='latin-1', index=False) # Exportando o arquivo para csv

# Salvando a Solução de "ipynb to py"

In [ ]:
tempo_final = time.time()
tempo_total = (float(tempo_final - tempo_inicial)/60)
print(f"Tempo total de execução: {tempo_total:,.1f} minutos" )